In [3]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from tqdm import tqdm
import time
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import os

url = 'https://ru.investing.com/crypto/bitcoin/btc-usd-historical-data?cid=1035793'

chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)
driver.get(url)
time.sleep(4)

# Увеличиваю окно браузера
driver.set_window_size(1800, 1000)

# Прокрутить
driver.execute_script("window.scrollTo(0, 650);") ### https://qastack.ru/programming/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

# Открываю поле воода даты
butom_date = 'DatePickerWrapper_input__MDvWH'
driver.find_element(By.CLASS_NAME, butom_date).click()
time.sleep(2)

# Ввод даты старта.
for try_n in range(1, 5):
    print(f'Попытка установки даты {try_n}')
    butom_close = 'NativeDateInput_root__wbgyP'  #'#__next > div > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__lRLYJ > main > div > div:nth-child(5) > div > div:nth-child(1) > div > div.flex.justify-between.items-end.mb-4 > div.flex.flex-row-reverse.desktop\:flex-row.gap-6 > div.historical-data_history-date-picker-wrapper__dDOuq > div.HistoryDatePicker_HistoryDatePicker__sjrlU > div:nth-child(1) > div > div:nth-child(1) > input[type=date]' #'//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'
    date_target = '01.12.2017'


    field = driver.find_element(By.CLASS_NAME, butom_close) #CSS_SELECTOR
    field = field.find_element(By.TAG_NAME, 'input')
    # field.clear()
    field.send_keys(date_target) # date_target + Keys.ENTER
    time.sleep(2)
    # Дата окончани не нужна. 
    if field.get_attribute('value')[:4] == date_target[-4:]:
        print('Утановленая дата =',  field.get_attribute('value'))
        print('Успешно')
        break
    else:
        print('Утановленая дата =',  field.get_attribute('value'))

# Применить фильтр.
butom_ = 'HistoryDatePicker_arrow-icon__NwxN4'  #'//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[2]/button'
driver.find_element(By.CLASS_NAME, butom_).click()
time.sleep(2)

# Проверить установился ли фильтр даты.
butom_ ='historical-data_history-date-picker-wrapper__dDOuq'
date_text = driver.find_element(By.CLASS_NAME, butom_).text

if date_text[:len(date_target)][-4:] == date_target[-4:]:
    print('Дата установлена верно')
else:
    print('Утановленная дата', date_text[:len(date_target)])
    1/0


# Прокрутить
driver.execute_script("window.scrollTo(0, 900);")

col_names = ['Дата', 'Цена', 'Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %']

# Пробую прочитать файл. 
try:
    df_parse = pd.read_excel('parse_df_investing.xlsx')
    pd.to_datetime(df_parse['Дата'], format='%d.%m.%Y')
    
    diff_day = 3
    date_old = pd.to_datetime(df_parse['Дата'], format='%d.%m.%Y').max()
    date_now_shift = datetime.datetime.today() + datetime.timedelta(days=diff_day)
    if date_old > date_now_shift:
        print(f'Старый файл отстаёт от сегодя меньше чем на {diff_day} дней. Новые данные не парсятся.')
    else:
        raise ValueError(f'Дата слишком слишком старая (больше {diff_day}). Придётся парсить данные')
        
    
except:
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # https://medium.com/analytics-vidhya/scraping-tables-from-a-javascript-webpage-using-selenium-beautifulsoup-and-pandas-cbd305ca75fe
    # beautifulsoup4 используемая библиотека

    tables = soup.find_all('table')
    dfs = pd.read_html(str(tables))
    for df_i in dfs:
        head_name = list(df_i)
        if head_name == ['Дата', 'Цена', 'Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %']:
            df_parse = df_i.copy()
            break

<ipython-input-3-916b2c1e32b5>:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)


Попытка установки даты 1
Утановленая дата = 0007-11-29
Попытка установки даты 2
Утановленая дата = 2017-11-28
Успешно
Дата установлена верно
1879


In [113]:
df_parse['Дата'] = pd.to_datetime(df_parse['Дата'], format='%d.%m.%Y')

In [114]:
def replace_num(str_):
    str_ = str(str_)
    if 'K' in str_:
        return int(str_.replace('K', '').replace(',', '')+'000')
    elif 'M' in str_:
        return int(str_.replace('M', '').replace(',', '')+'000000')
    else:
        try:
            return int(str_.replace(',', ''))
        except:
            return 0
        
df_parse['value_correct'] = df_parse['Объём'].apply(replace_num)

In [115]:
def replace_symb(str_):
    str_ = float(str(str_).replace('.', '').replace(',', '.'))
    return str_

In [116]:
for col in ['Цена', 'Откр.', 'Макс.', 'Мин.', 'value_correct']:
    print(col)
    df_parse[col] = df_parse[col].apply(replace_symb)


driver.close()

Цена
Откр.
Макс.
Мин.
value_correct


# Parce google trands

In [154]:
url = '''https://trends.google.ru/trends/?geo=KZ'''

chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)
driver.get(url)

<ipython-input-154-7ef9c33dd142>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), chrome_options=chrome_options)


In [155]:
# # Ввод поискового запроса
butom_close = '//*[@id="input-254"]'
find_data = 'buy bitcoin' 

field = driver.find_element(By.XPATH, butom_close)
field.clear()
field.send_keys(find_data + Keys.ENTER) # date_target + Keys.ENTER
time.sleep(2)
# Дата окончани не нужна. 

In [156]:
# Открываю список регионов
butom_ = '//*[@id="compare-pickers-wrapper"]/div/hierarchy-picker[1]/ng-include/div[1]'
find_data = 'buy bitcoin' 
set_local = 'По всему миру'
set_local = 'Казахстан'


field = driver.find_element(By.XPATH, butom_).click()
time.sleep(1)


In [157]:
# Ввожу что мне нужны данные по всему миру
butom_ = '//*[@id="input-8"]'
set_local = 'По всему миру'


field = driver.find_element(By.XPATH, butom_)
field.clear() #clear
field.send_keys(set_local + Keys.ENTER)
time.sleep(3)


In [158]:
# Выбираю из выподающего списка
butom_ = '//*[@id="ul-8"]/li/md-autocomplete-parent-scope/div'

find_data = 'buy bitcoin' 
set_local = 'По всему миру'


field = driver.find_element(By.XPATH, butom_).click()
time.sleep(3)


In [175]:
# Открываю список периодов
butom_ = 'compare-picker ng-pristine ng-untouched ng-valid ng-not-empty'.replace(' ', '.')

field = driver.find_element(By.CLASS_NAME, butom_).click()
time.sleep(3)

# Поиск по нескольким классам.
# https://stackoverflow.com/questions/60534244/how-to-locate-an-element-with-multiple-classnames-using-selenium-and-python

In [178]:
# Нажимаю произвольный период.
butom_ = 'custom-date-picker-select-option md-ink-ripple'.replace(' ', '.')

driver.find_elements(By.CLASS_NAME, butom_)[-1].click()

In [180]:
# Выбираю дату начала.
butom_ = 'md-datepicker-input'


field = driver.find_element(By.CLASS_NAME, butom_)
field.clear()
field.send_keys(date_target)

In [181]:
# Нажимаю ок

n_d = 3 # number_decision

butom_ = 'custom-date-picker-dialog-button md-button md-ink-ripple'.replace(' ', '.')

field = driver.find_elements(By.CLASS_NAME, butom_)
field[-1].click()

time.sleep(1)

In [184]:
# Скачать данные

n_d = 1 # number_decision

butom_ = 'widget-actions-item export'.replace(' ', '.')

field = driver.find_elements(By.CLASS_NAME, butom_)[0].click()
time.sleep(1)

In [185]:
driver.close()

In [ ]:
# Находим последний загруженнй файл
dowlend_path = 'C:/Users/Kirill/Downloads'
list_paths = []

for f_name in os.listdir(dowlend_path):
    dowlend_file_p = dowlend_path + '//' + f_name
    list_paths.append(dowlend_file_p)

latest_file = max(list_of_files, key=os.path.getctime)
# Получить название последнего файла
# https://stackoverflow.com/questions/39327032/how-to-get-the-latest-file-in-a-folder

# Получить дату изменения файла
# https://stackoverflow.com/questions/237079/how-do-i-get-file-creation-and-modification-date-times

In [ ]:
pd.read_csv(latest_file)